In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

## ■ Similarity Filtering + Counting

In [ ]:
cd "../../Data/Preprocessed_data"

In [ ]:
!pip3 install -U spaCy
!python -m spaCy download en_core_web_lg

__* en_core_web_lg__
> 영어를 위한 훈련된 파이프라인입니다. CPU에 최적화되어 있으며 ok2vec, tagger, parser, senter, ner, attribute_ruler, lemmatizer와 같은 구성 요소를 포함합니다. 파일 크기는 en_core_web_md와 비교하여 741MB입니다. Spacy에서 제공하는 가장 큰 영어 모델입니다.

In [5]:
# 필요 라이브러리 선언
import pandas as pd
import spacy
import ast

### - spaCy와 similarity 함수: cosine similarity 기반

In [6]:
# spacy의 자연어 처리 기능을 사용하여 키워드 간의 유사도를 계산
# 입력값으로 두 개의 키워드 문자열과 spacy 객체를 받으며, 출력값은 두 문장 간의 유사도로 계산된 float
def similarity_between_keywords(keyword1, keyword2, nlp):
    doc1 = nlp(keyword1)
    doc2 = nlp(keyword2)
    
    # 두 Docs의 벡터가 모두 존재할 때만 유사도 계산, 그렇지 않으면 0 반환
    if doc1.has_vector and doc2.has_vector:
        return doc1.similarity(doc2)
    else:
        return 0.0

# 주어진 키워드 목록을 순회하면서 유사한 구문을 필터링
def filter_similar_keywords(keywords, threshold=0.7):
    # 키워드 목록이 비어있는 경우 바로 빈 리스트 반환
    if len(keywords) == 0:
        return []
    
    nlp = spacy.load("en_core_web_lg")
    filtered_keywords = []

    # 순서대로 키워드를 검토하면서 기존 대표 키워드들(filtered_keywords)과 비교하여 유사도가 threshold보다 높은 키워드가 없으면 그 키워드를 대표 키워드 목록에 추가
    for keyword in keywords:
        # 키워드 앞뒤 공백 있을 시 제거, 키워드 없을 경우 스킵
        keyword = keyword.strip()
        if not keyword:
            continue
    
        if not any([similarity_between_keywords(keyword, existing_keyword, nlp) > threshold for existing_keyword in filtered_keywords]):
            filtered_keywords.append(keyword)
    
    return filtered_keywords

# keywords = ['short bitter finish', 'golden colour', 'herbal aroma', 'good standard lager', 'standard lager', 'herbal aroma taste', 'godd white head', 'golden colour godd']
# filtered_keywords = filter_similar_keywords(keywords)
# print(filtered_keywords)


---

## ■ 리뷰별 동의어, 유의어 필터링 이후 단순 카운팅
- 리뷰별로 해당 키워드가 몇 개 나왔는지 파악
- 해당 맥주에 대한 키워드 집계를 나타냄으로써 맥주 리뷰 가독성 Up

In [ ]:
# 특정 맥주 csv 파일 불러오기
Filtered_df = pd.read_csv('pp_selected_reviews.csv')

# 문자열 형태의 리스트를 실제 리스트로 변환 후, filter_similar_keywords 적용
Filtered_df['Filtered_Keywords'] = Filtered_df['Keywords'].apply(lambda x: filter_similar_keywords(ast.literal_eval(x)))

Filtered_df

In [7]:
# # 필터링된 키워드 목록을 CSV 파일로 저장
# Filtered_df.to_csv('8_Wired_iStout_df_filtered.csv', index=False)

In [9]:
Filtered_df = pd.read_csv('8_Wired_iStout_df_filtered.csv')
Filtered_df

,Review,Beer_name,MultinomialNB_label,Keywords,Filtered_Keywords
0,"Inky black beer , small head ... sweet molasse...",8 Wired iStout,Positive,"['sweet molasses aroma', 'mocha taste', 'chocc...","['sweet molasses aroma', 'choccy mocha', 'inky..."
1,Draft. Black beer with a tan head. Chocolate a...,8 Wired iStout,Positive,"['toffee flavor', 'toffee linger', 'head', 'to...","['toffee flavor', 'head', 'coffee', 'draft bla..."
2,On tap. Poured black color with a tan head. Ar...,8 Wired iStout,Positive,"['roasted malt', 'dark chocolate coffee', 'fin...","['roasted malt', 'dark chocolate coffee', 'fin..."
3,"Roasted dark chocolate aroma, beautiful black ...",8 Wired iStout,Positive,"['black licorice flavor', 'roast', 'roasted da...","['black licorice flavor', 'roast', 'coloring h..."
4,"Old rating - 500ml bottle, 10%. Pours black wi...",8 Wired iStout,Positive,"['500ml bottle 10', 'malt medium body', 'taste...","['500ml bottle 10', 'malt medium body', 'taste..."
...,...,...,...,...,...
397,Bottle from Vinens Verden. Nice tan head with ...,8 Wired iStout,Positive,"['vinens verden', 'vinens verden nice', 'nice ...","['vinens verden', 'vinens verden nice', 'nice ..."
398,Pours totally black with a very brown head. Ar...,8 Wired iStout,Positive,"['caramel', 'taste', 'black brown head', 'roas...","['caramel', 'taste', 'black brown head', 'head..."
399,Tried again Bottle: This beer sets the standar...,8 Wired iStout,Positive,"['standard australasian imperial', 'best beer'...","['standard australasian imperial', 'best beer'..."
400,my non ---Rated didn't use of goods and servic...,8 Wired iStout,Negative,"['buddy', 'app', 'non rated', 'benefit used be...","['buddy', 'app', 'non rated', 'benefit used be..."


In [12]:
from collections import Counter
import ast

def count_all_keywords(dataframe):
    # 문자열로 표현된 리스트를 실제 리스트로 변환
    dataframe['Keywords_List'] = dataframe['Filtered_Keywords'].apply(lambda x: ast.literal_eval(x))

    # 모든 키워드 리스트를 하나로 합칩니다.
    all_keywords = dataframe['Keywords_List'].sum()

    keyword_counts = Counter(all_keywords)
    sorted_keyword_counts = keyword_counts.most_common()  # 키워드 빈도가 많은 순으로 내림차순으로 정렬한다.

    # 변환된 'Keywords_List' 컬럼을 삭제합니다.
    dataframe.drop('Keywords_List', axis=1, inplace=True)

    return sorted_keyword_counts

# 모든 인덱스의 'keywords' 컬럼에 있는 단어들을 카운트
sorted_all_keyword_counts = count_all_keywords(Filtered_df)

In [13]:
print(*sorted_all_keyword_counts, sep='\n')

('taste', 61)
('head', 48)
('bottle', 46)
('roasted malt', 40)
('chocolate', 36)
('aroma', 34)
('coffee', 27)
('licorice', 23)
('dark chocolate', 22)
('flavor', 17)
('vanilla', 16)
('roasted malts', 15)
('liquorice', 15)
('caramel', 14)
('black', 13)
('malt', 9)
('cocoa', 9)
('pours', 9)
('dark fruit', 9)
('flavour', 9)
('tastes', 8)
('beer', 8)
('black pour', 7)
('beer buddy', 7)
('molasses', 7)
('500ml bottle', 7)
('stout', 7)
('tan head', 7)
('roast', 6)
('pours black', 6)
('medium bitterness', 6)
('aromas', 6)
('milk chocolate', 6)
('roasted coffee', 6)
('good beer', 6)
('brown head aroma', 6)
('brown head', 5)
('stouts', 5)
('beige head', 5)
('black brown head', 5)
('imperial stout', 5)
('bottle pours', 5)
('nice', 5)
('jet black', 5)
('tan head aroma', 4)
('bitter dark chocolate', 4)
('good', 4)
('head aroma chocolate', 4)
('bitter finish', 4)
('dark fruits', 4)
('bourbon', 4)
('good stout', 4)
('iphone', 4)
('bitter chocolate', 4)
('head aroma', 4)
('espresso', 4)
('bottle black

#### __🤔결과:__ 
- 1721개의 빈도 카테고리가 나옴. 
#### __🧐결론:__ 
- 맥주와 관련있는 키워드라고 볼 수 있으나 해당 맥주의 특징이라고는 볼 수 없는 키워드들이 다수 포착
- <u>리뷰의 특징을 잘 묘사하지 못하는 키워드들</u>이 높은 랭크를 차지함과 동시에 카테고리 뻥튀기의 주범이 됨
- 주로 단일 단어에서 이런 양상을 띔
- 예) taste, head, bottle, ...


<hr style="border: none; border-top: 2px dashed gray;"/>

### - 동의어 그룹핑 Count 방법 1 

In [14]:
import spacy

nlp = spacy.load('en_core_web_lg')

def merge_similar_keywords(sorted_keyword_counts, similarity_threshold=0.7):
    merged_keywords = []
    
    for keyword, count in sorted_keyword_counts:
        doc1 = nlp(keyword)
        
        # 유사한 키워드를 찾으면 그 인덱스를 계속 업데이트
        similar_index = -1
        
        for i, (merged_keyword, merged_count) in enumerate(merged_keywords):
            doc2 = nlp(merged_keyword)
            
            if doc1.similarity(doc2) > similarity_threshold:
                similar_index = i
                break
        
        # 유사한 키워드가 있다면 빈도수를 합침
        if similar_index >= 0:
            merged_keywords[similar_index] = (merged_keywords[similar_index][0], merged_keywords[similar_index][1] + count)
        else:
            # 유사한 키워드가 없다면 새 키워드를 추가
            merged_keywords.append((keyword, count))

    return merged_keywords

merged_keyword_counts = merge_similar_keywords(sorted_all_keyword_counts)
print(merged_keyword_counts)

C:\Users\Gyeom\AppData\Local\Temp\ipykernel_19200\1687830486.py:17: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  if doc1.similarity(doc2) > similarity_threshold:


[('taste', 234), ('head', 161), ('bottle', 113), ('roasted malt', 168), ('chocolate', 194), ('coffee', 80), ('licorice', 42), ('black', 116), ('pours', 23), ('dark fruit', 97), ('beer', 61), ('black pour', 17), ('molasses', 7), ('stout', 16), ('roast', 14), ('medium bitterness', 34), ('aromas', 30), ('brown head aroma', 53), ('imperial stout', 24), ('nice', 30), ('jet black', 10), ('good', 24), ('bitter finish', 37), ('iphone', 7), ('lots', 8), ('body', 16), ('color', 13), ('like', 11), ('excellent stout', 10), ('istout', 3), ('smoke', 4), ('syrupy', 4), ('nose', 9), ('notes', 12), ('piney hops', 9), ('rated', 9), ('bit', 11), ('far', 3), ('appearance', 2), ('long aftertaste', 6), ('moderate carbonation', 13), ('big chocolate', 27), ('brisk bitterness', 4), ('tap', 6), ('zealand', 2), ('keg', 3), ('brewdog online', 3), ('pretty', 5), ('hop', 13), ('500ml', 2), ('works', 2), ('ris', 2), ('light caramel', 20), ('pitch', 5), ('delicious', 18), ('rated beer buddy', 5), ('oil', 5), ('strong

In [19]:
print(*merged_keyword_counts, sep="\n")

('taste', 234)
('head', 161)
('bottle', 113)
('roasted malt', 168)
('chocolate', 194)
('coffee', 80)
('licorice', 42)
('black', 116)
('pours', 23)
('dark fruit', 97)
('beer', 61)
('black pour', 17)
('molasses', 7)
('stout', 16)
('roast', 14)
('medium bitterness', 34)
('aromas', 30)
('brown head aroma', 53)
('imperial stout', 24)
('nice', 30)
('jet black', 10)
('good', 24)
('bitter finish', 37)
('iphone', 7)
('lots', 8)
('body', 16)
('color', 13)
('like', 11)
('excellent stout', 10)
('istout', 3)
('smoke', 4)
('syrupy', 4)
('nose', 9)
('notes', 12)
('piney hops', 9)
('rated', 9)
('bit', 11)
('far', 3)
('appearance', 2)
('long aftertaste', 6)
('moderate carbonation', 13)
('big chocolate', 27)
('brisk bitterness', 4)
('tap', 6)
('zealand', 2)
('keg', 3)
('brewdog online', 3)
('pretty', 5)
('hop', 13)
('500ml', 2)
('works', 2)
('ris', 2)
('light caramel', 20)
('pitch', 5)
('delicious', 18)
('rated beer buddy', 5)
('oil', 5)
('strong chocolate', 7)
('really', 9)
('one', 2)
('nicely', 3)
('v

<hr style="border: none; border-top: 2px dashed gray;"/>

### - 동의어 그룹핑 Count 방법 2: Mapping

In [25]:
import spacy

nlp = spacy.load('en_core_web_lg')

def merge_similar_keywords(sorted_keyword_counts, similarity_threshold=0.8):
    merged_keywords = []
    
    for keyword, count in sorted_keyword_counts:
        doc1 = nlp(keyword)
        
        # 유사한 키워드를 찾으면 그 인덱스를 계속 업데이트
        similar_index = -1
        
        for i, (merged_keyword, merged_count) in enumerate(merged_keywords):
            doc2 = nlp(merged_keyword.split(", ")[0])
            
            if doc1.similarity(doc2) > similarity_threshold:
                similar_index = i
                break
        
        # 유사한 키워드가 있다면 빈도수를 합침
        if similar_index >= 0:
            existing_keywords = merged_keywords[similar_index][0]
            updated_keywords = f"{existing_keywords}, {keyword}"
            updated_count = merged_keywords[similar_index][1] + count
            merged_keywords[similar_index] = (updated_keywords, updated_count)
        else:
            # 유사한 키워드가 없다면 새 키워드를 추가
            merged_keywords.append((keyword, count))

    return merged_keywords

merged_keyword_counts = merge_similar_keywords(sorted_all_keyword_counts)
print(*merged_keyword_counts, sep="\n")


C:\Users\Gyeom\AppData\Local\Temp\ipykernel_23192\852691680.py:17: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  if doc1.similarity(doc2) > similarity_threshold:


('taste, flavor, flavour, tastes, chocolate taste, espresso taste cocoa, coffee alcohol taste, malt coffee taste, taste coffee, roast taste, alcohol vanilla taste, bourbon taste dark, mocha taste, chocolate vanilla taste, liquorice aroma taste, choc taste, coffee taste, taste licorice chocolate, taste roasted malts, taste licorice, taste overall beer, licorice flavor, taste pleasant rich, taste coffe, great roasted malts aroma taste, strong taste, stout excellent taste, aroma taste, toasted coffe taste, taste hopped, pleasant taste, choclate flavor licorice, barry smokey taste', 126)
('head, brown head, beige head, foamy beige head, tanned head, little head, head aromas, head lacing, coloring head, head suds lacing, head nose shows, black head, fudgey beige head, head strong, finger brown head, head massive, head roasted, beige head gelatine, head decent, head bitter, slight mocha head, colour head, lacy head, snifter mocha head, head dissapears quickly, chocolate head, head coffee, he